## KAKAOMAP 상세보기 리뷰 크롤링 코드 ##

In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
import sys

import warnings
warnings.filterwarnings('ignore')


In [11]:
## CSV 파일 불러오기
df = pd.read_csv('kakaomap_yeonnam_restaurant_moreview_list.csv') #, encoding='cp949')
df

,사업장명,소재지전체주소,도로명전체주소,좌표정보X(EPSG5174),좌표정보Y(EPSG5174),상세보기url
0,사르르,서울특별시 마포구 연남동 383-100,"서울특별시 마포구 연희로1길 63, 2층 (연남동)",193442.684557,451030.233681,NaN
1,빨강꼬치&꼬떡상회,서울특별시 마포구 연남동 390-71,"서울특별시 마포구 동교로38길 13, 지1층 (연남동)",193303.524530,451089.391094,NaN
2,지라파,서울특별시 마포구 연남동 373-12,"서울특별시 마포구 연남로 8, 1층 (연남동)",193169.497265,450929.132987,NaN
3,돈토키,서울특별시 마포구 연남동 390-28,"서울특별시 마포구 동교로38길 27-19, 지층 B02호 (연남동)",193388.640488,451076.653833,NaN
4,에스유 치즈카페,서울특별시 마포구 연남동 225-29,"서울특별시 마포구 성미산로 175, 지하1층 (연남동)",193333.879086,451308.487923,NaN
...,...,...,...,...,...,...
795,연남제비,서울특별시 마포구 연남동 228-5,"서울특별시 마포구 성미산로 186 (연남동, 1층)",193391.074652,451202.033340,https://place.map.kakao.com/2045155753
796,유키모찌 연남점,서울특별시 마포구 연남동 260-28,"서울특별시 마포구 동교로 240 (연남동,1층)",193255.606492,451004.993797,https://place.map.kakao.com/913195358
797,항저우 샤롱바오,서울특별시 마포구 연남동 568-23,"서울특별시 마포구 동교로27길 41 (연남동,(1층))",192990.763157,450825.627592,https://place.map.kakao.com/961577779
798,뭉텅 연남점,서울특별시 마포구 연남동 228-1,"서울특별시 마포구 성미산로 190 (연남동, 1층)",193406.589452,451183.710005,https://place.map.kakao.com/388449263


In [12]:
## DF에서 NaN 또는 빈 문자열을 제외한 값만 리스트로 변환
url_list = df['상세보기url'].dropna().tolist()  # NaN 제거
url_list = [url for url in url_list if url.strip()]  # 빈 문자열 제거

# # 결과 출력
# print(url_list)

In [13]:
## 생성한 리스트에 빈 문자열 또는 NaN 값이 있는지 검증
has_empty_or_nan = any(item == "" or item is None or (isinstance(item, float) and np.isnan(item)) for item in url_list)
print("리스트에 빈 문자열 또는 NaN이 포함되어 있나요?", has_empty_or_nan)

리스트에 빈 문자열 또는 NaN이 포함되어 있나요? False


In [14]:
## 리스트 갯수(길이) 확인
len(url_list)

691

In [15]:
## 스크래핑 준비

## 리뷰 저장용 DF 생성
review_df = pd.DataFrame(columns=['사업장명',
                                  '분류',
                                  '카카오맵ID',
                                  '리뷰어이름',
                                  '리뷰어레벨',
                                  '후기개수',
                                  '별점평균',
                                  '팔로워수',
                                  '작성일',
                                  '별점',
                                  '리뷰내용',
                                  '사진URL'
                                  ])

## 스크래핑용 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
# time.sleep(2)

In [17]:
## 카카오맵 검색 가게 후기 스크래핑 ###

## 리뷰 스크래핑 함수
def all_review_scraping(temp_rev, rev_cnt):
    global store_name, store_category, store_id
    temp_rev_list = []
    # temp_rev = review_group[0].contents[rev_cnt]

    # 리뷰어 이름
    reviewer_name_tag = temp_rev.select_one('.name_user')
    reviewer_name = reviewer_name_tag.contents[1].strip() if reviewer_name_tag and len(reviewer_name_tag.contents) > 1 else "N/A"
    
    # 리뷰어 레벨
    reviewer_level_tag = temp_rev.select_one('.txt_badge')
    reviewer_level = reviewer_level_tag.text.replace('레벨', '').strip() if reviewer_level_tag else "N/A"
    
    # 후기, 별점평균, 팔로워 수 (list_detail 활용)
    review_stats = {}
    list_detail = temp_rev.select_one('.list_detail')
    if list_detail:
        items = list_detail.find_all('li')
        for item in items:
            text = item.text.strip().split(' ')
            if len(text) >= 2:
                key = text[0] # '후기', '별점평균', '팔로워'
                value = text[1] # '162', '4.1', '5'
                review_stats[key] = value

    # 리뷰 작성일
    review_date_tag = temp_rev.select_one('.txt_date')
    review_date = review_date_tag.text.strip() if review_date_tag else "N/A"

    # 별점 (screen_out 클래스 중 두 번째에 텍스트 별점이 있음)
    rating_tag = temp_rev.select('.starred_grade .screen_out')
    # 두 번째 screen_out 태그에 실제 숫자가 들어있음 (별점 1.0)
    rating = rating_tag[1].text.strip() if len(rating_tag) > 1 else "N/A"

    # 리뷰 내용
    review_content_tag = temp_rev.select_one('.desc_review')
    if review_content_tag:
        # <p class="desc_review">텍스트 내용 <span class="btn_more">더보기</span></p>
        # p 태그의 첫 번째 자식 요소가 우리가 원하는 텍스트일 가능성이 높음.
        review_content_parts = [
            str(c) for c in review_content_tag.contents if c.name != 'span' and isinstance(c, str)
        ]
        review_content = ''.join(review_content_parts).replace('\n', '').strip()    
    else:
        review_content = "N/A"

    temp_rev_list.append(store_name)        # 가게 이름
    temp_rev_list.append(store_category)    # 가게 분류
    temp_rev_list.append(store_id)          # 카카오맵 ID
    temp_rev_list.append(reviewer_name)     # 리뷰어 이름
    temp_rev_list.append(reviewer_level)    # 리뷰어 레벨
    temp_rev_list.append(review_stats.get('후기', 'N/A'))       # 후기 개수
    temp_rev_list.append(review_stats.get('별점평균', 'N/A'))   # 별점 평균
    temp_rev_list.append(review_stats.get('팔로워', 'N/A'))     # 팔로워 수
    temp_rev_list.append(review_date)     # 리뷰 작성일
    temp_rev_list.append(rating)     # 별점
    temp_rev_list.append(review_content)    # 리뷰 내용

    # 리뷰 사진 URL
    photo_urls = []
    # .list_photo 클래스를 가진 ul 태그 안의 모든 img 태그를 찾습니다.
    # CSS Selector: .list_photo img
    img_tags = temp_rev.select('.list_photo img.img_g')

    if img_tags:
        for img_tag in img_tags:
            # img 태그의 'src' 속성 값을 가져옵니다.
            if 'src' in img_tag.attrs:
                photo_url = img_tag['src']
                photo_urls.append(photo_url)

        temp_rev_list.append(photo_urls)                
    else:
        temp_rev_list.append("N/A")
            
    return temp_rev_list

print("### 리뷰 수집을 시작합니다. ###")

### 코드 본문 시작점 ###
try:
    store_counter = 1
    for url in url_list:
        print(f"\n--- 새로운 가게 처리 시작: {url} ---")
        driver.get(url)
        time.sleep(2)

        ### '후기' 탭 유무 확인 및 클릭
        print("페이지 로드 중... '후기' 탭을 확인합니다.")
        review_tab_found_and_clicked = False
        potential_review_tabs = driver.find_elements(By.CSS_SELECTOR, 'a.link_tab[href="#comment"]')
        
        if potential_review_tabs:
            # 요소가 발견되었다면, 이제 클릭 가능한지 확인하고 클릭
            try:
                review_tab_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable(potential_review_tabs[0]) # 첫 번째 요소를 대상으로
                )
                review_tab_button.click()
                print("'후기' 탭을 클릭했습니다.")
                review_tab_found_and_clicked = True
                time.sleep(1) # '후기' 탭 클릭 후 페이지 내용이 업데이트될 시간

            except Exception as e:
                print(f"$$$ '후기' 탭은 찾았으나 클릭할 수 없습니다.: {e} 건너뛰고 다음 가게로 넘어갑니다. $$$\n")
                continue # 다음 가게로 넘어가기

        else:
            print(f"$$$ '후기' 탭이 없습니다. 다음 가게로 넘어갑니다. $$$\n")
            continue # 다음 가게로 넘어가기

        ### '후기' 탭이 열렸으면 리뷰 내용에서 '더보기' 버튼 확장
        if review_tab_found_and_clicked:
            print("'더보기' 버튼을 확인하고 확장합니다.")
            clicked_more_buttons_count = 0
            
            # '더보기' 버튼 존재 여부를 빠르게 확인하고, 없으면 바로 다음 과정으로 이동
            potential_more_buttons = driver.find_elements(By.CSS_SELECTOR, 'ul.list_review span.btn_more')
            
            more_buttons_to_click = [
                btn for btn in potential_more_buttons if btn.text.strip() == '더보기'
            ]

            if not more_buttons_to_click:
                print("'더보기' 버튼이 없어 다음 작업을 진행합니다.")
                # '더보기' 버튼이 없으므로, 클릭 과정 없이 바로 다음 작업으로 진행합니다.
            else:
                print(f"총 {len(more_buttons_to_click)}개의 '더보기' 버튼을 확장")
                
                for i, button_element in enumerate(more_buttons_to_click):
                    try:
                        # 클릭 가능한 상태인지 확인 (JavaScript 클릭에 대한 추가적인 방어)
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(button_element))
                        
                        driver.execute_script("arguments[0].scrollIntoView(true);", button_element)
                        driver.execute_script("arguments[0].click();", button_element)
                        
                        # print(f"  - [{i+1}/{len(more_buttons_to_click)}] '더보기' 버튼 클릭 성공.")
                        clicked_more_buttons_count += 1
                        time.sleep(0.1) # 클릭 후 DOM 업데이트를 위한 짧은 대기
                        
                    except Exception as click_error:
                        print(f"  - [{i+1}/{len(more_buttons_to_click)}] 버튼 클릭 실패: {click_error}")

                # print(f"총 {clicked_more_buttons_count}개의 '더보기' 버튼을 성공적으로 클릭했습니다.")
                
        # 4. 모든 '더보기' 버튼 확장 후 최종 HTML 파싱 및 데이터 추출
        # print(f"--- {url} 가게의 모든 '더보기' 버튼 클릭 작업 완료. 최종 데이터 추출을 준비합니다. ---")
        
        ### 파싱 및 리뷰 수집 시작
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # time.sleep(0.2)        

        ## 가게 이름 가져오기
        title_info_tag = soup.select('#mainContent > div.top_basic > div.info_main > div.unit_info')
        store_name = title_info_tag[0].contents[0].contents[1]
        store_category = title_info_tag[0].contents[2].contents[1]
        store_id = url.split('/')[-1]

        ## 현재 크롤링 중인 가게 이름 표시 ##
        print("## no.", store_counter, " 사업장이름:",store_name ,"/ 분류:",store_category, "/ 카카오맵ID :",store_id , "##")

        ## 리뷰 그룹 가져오기
        review_group = soup.select('#mainContent > div.main_detail > div.detail_cont > div.section_comm.section_review > div.group_review > ul')

        rev_cnt = 0     # 리뷰 수 카운터
        start_perf_counter = time.perf_counter()    # 시간 측정용
        while review_group:
            if rev_cnt >= len(review_group[0].contents)-1:
                break

            temp_rev = review_group[0].contents[rev_cnt]
            time.sleep(0.1)
            temp_rev_list = all_review_scraping(temp_rev, rev_cnt)   

            print(temp_rev_list)    # 스크래핑한 리뷰 리스트 출력
            review_df.loc[len(review_df)] = temp_rev_list   # 사전 지정 DF에 추가
            rev_cnt += 1
            # time.sleep(0.5)

        end_perf_counter = time.perf_counter()
        elapsed_perf_time = end_perf_counter - start_perf_counter
        print(f"===== 걸린 시간 : {elapsed_perf_time:.2f} 초 =====") # 걸린 시간 표시

        store_counter += 1

except Exception as main_error:
    print(f"\n스크립트 실행 중 치명적인 오류 발생: {main_error}")

finally:
    driver.quit()
    print('** 리뷰 수집 완료. WebDriver를 종료합니다. **')


### 리뷰 수집을 시작합니다. ###

--- 새로운 가게 처리 시작: https://place.map.kakao.com/633908761 ---
페이지 로드 중... '후기' 탭을 확인합니다.
'후기' 탭을 클릭했습니다.
'더보기' 버튼을 확인하고 확장합니다.
'더보기' 버튼이 없어 다음 작업을 진행합니다.
## no. 1  사업장이름: EP 커피N바 / 분류: 양식 / 카카오맵ID : 633908761 ##
['EP 커피N바', '양식', '633908761', '피트조아', '8', '7', '4.4', '0', '2023.07.19.', '5.0', '최고입니다', ['//img1.kakaocdn.net/cthumb/local/C280x280.q50/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocal%2FkakaomapPhoto%2Freview%2F6bd3293cb6e38c0d8fd44c375b8207dfb2632fc2%3Foriginal']]
['EP 커피N바', '양식', '633908761', 'ppp', '17', '33', '3.6', '0', '2023.07.02.', '5.0', '맛은 물론이고 너무나 유쾌한 접객. 단골해야 해요', ['//img1.kakaocdn.net/cthumb/local/C280x280.q50/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocal%2FkakaomapPhoto%2Freview%2Fe3534eb150c51822c216dea4cb8d143ece155c1a%3Foriginal']]
===== 걸린 시간 : 0.22 초 =====

--- 새로운 가게 처리 시작: https://place.map.kakao.com/821235252 ---
페이지 로드 중... '후기' 탭을 확인합니다.
'후기' 탭을 클릭했습니다.
'더보기' 버튼을 확인하고 확장합니다.
총 6개의 '더보기' 버튼을 확장
## no. 2  사업장이름: 순대일번지 / 분류: 순대 / 카카오맵ID :

In [46]:
review_df

,사업장명,분류,카카오맵ID,리뷰어이름,리뷰어레벨,후기개수,별점평균,팔로워수,작성일,별점,리뷰내용,사진URL
0,EP 커피N바,양식,633908761,피트조아,8,7,4.4,0,2023.07.19.,5.0,최고입니다,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
1,EP 커피N바,양식,633908761,ppp,17,33,3.6,0,2023.07.02.,5.0,맛은 물론이고 너무나 유쾌한 접객. 단골해야 해요,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
2,순대일번지,순대,821235252,댕,4,2,1.5,1,2025.05.07.,2.0,굳이... 평점이 왜 높은지 모르겠는... 밥도 설익고 식었고..국물이 그렇게 ...,N/A
3,순대일번지,순대,821235252,리브흐,21,47,5,0,2025.03.20.,5.0,가게이름이 납득이갑니다안짜고요 양많아요양념으로 범벅하는 순대국 아니라 좋아요,N/A
4,순대일번지,순대,821235252,마르스,32,80,3.9,23,2025.02.27.,4.0,순대국특 먹었어요.다대기 들어있으니까 일단 먹어보고 싱거우면 새우젖으로 간해서 드세...,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
...,...,...,...,...,...,...,...,...,...,...,...,...
7554,코리아식당,한식,15742085,혀니,34,142,2.8,8,2021.07.21.,5.0,사장님도 친절하시고 시원시원하시고연남동에서 잘 찾아볼 수 없는 orthodox 한식...,N/A
7555,코리아식당,한식,15742085,Rena,23,66,3.4,4,2021.03.21.,1.0,홀에서 서빙하는 남자분 예의가 없음손님한테 명령조의 반말함,N/A
7556,코리아식당,한식,15742085,5_4_1_2,12,19,3.9,0,2021.03.12.,5.0,밑반찬도 자주 바뀌고 맛있고 저녁에 술판나도 당당히 밥 먹을수 있는곳. 그리고 묘하...,N/A
7557,코리아식당,한식,15742085,눈의띠네,38,669,3.9,17,2020.10.29.,3.0,찾아갈 맛집은 아니지만 무난한 밥집늦은 시간 연남에서 밥이 고플 때 거의 유일한 선택지,N/A


In [45]:
## csv 파일로 저장
review_df.to_csv("kakaomap_yeonnam_review_scrap.csv", index=False, encoding="utf-8-sig")


In [47]:
## 정제용 DF 복제
filtered_df = review_df.copy()
filtered_df.head(10)

,사업장명,분류,카카오맵ID,리뷰어이름,리뷰어레벨,후기개수,별점평균,팔로워수,작성일,별점,리뷰내용,사진URL
0,EP 커피N바,양식,633908761,피트조아,8,7,4.4,0,2023.07.19.,5.0,최고입니다,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
1,EP 커피N바,양식,633908761,ppp,17,33,3.6,0,2023.07.02.,5.0,맛은 물론이고 너무나 유쾌한 접객. 단골해야 해요,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
2,순대일번지,순대,821235252,댕,4,2,1.5,1,2025.05.07.,2.0,굳이... 평점이 왜 높은지 모르겠는... 밥도 설익고 식었고..국물이 그렇게 ...,N/A
3,순대일번지,순대,821235252,리브흐,21,47,5,0,2025.03.20.,5.0,가게이름이 납득이갑니다안짜고요 양많아요양념으로 범벅하는 순대국 아니라 좋아요,N/A
4,순대일번지,순대,821235252,마르스,32,80,3.9,23,2025.02.27.,4.0,순대국특 먹었어요.다대기 들어있으니까 일단 먹어보고 싱거우면 새우젖으로 간해서 드세...,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
5,순대일번지,순대,821235252,여행 지킴이,32,116,4.7,1,2025.02.10.,5.0,여행타임tv를 운영하는 크리에이터입니다서울시내 순대국집 100군데를 다녀본바로 여긴...,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
6,순대일번지,순대,821235252,인혜,3,1,5,0,2025.02.08.,5.0,연남동에서 순대국 처음 먹어봤는데 생각 이상으로 국물이 깔끔하고 고기도 많았습니다!...,[//img1.kakaocdn.net/cthumb/local/C280x280.q50...
7,순대일번지,순대,821235252,동환,21,29,4.6,2,2025.02.08.,5.0,주에 최소 한번은 순대국 수혈해야 살수있는사람인데집근처에 있길래 한번가봤는데 너무 ...,N/A
8,순대일번지,순대,821235252,박덕동 곽종필,8,6,3.8,0,2025.02.07.,3.0,김치가 너무 달고 순대도너무적었음,N/A
9,순대일번지,순대,821235252,맥아리장군,16,21,1.8,1,2025.01.04.,1.0,제가 딱 한마디로 표현하겠습니다 여기는 순대국을 단 한번도 안 먹어본 사장님이 운영...,N/A


In [48]:
## 사진URL 행의 리스트 -> 문자열 전환

# 셀의 값이 리스트인지 확인하고, 리스트이면 join()으로 문자열 변환, 아니면 원래 값 유지
filtered_df['사진URL'] = filtered_df['사진URL'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
filtered_df.head(15)

,사업장명,분류,카카오맵ID,리뷰어이름,리뷰어레벨,후기개수,별점평균,팔로워수,작성일,별점,리뷰내용,사진URL
0,EP 커피N바,양식,633908761,피트조아,8,7,4.4,0,2023.07.19.,5.0,최고입니다,//img1.kakaocdn.net/cthumb/local/C280x280.q50/...
1,EP 커피N바,양식,633908761,ppp,17,33,3.6,0,2023.07.02.,5.0,맛은 물론이고 너무나 유쾌한 접객. 단골해야 해요,//img1.kakaocdn.net/cthumb/local/C280x280.q50/...
2,순대일번지,순대,821235252,댕,4,2,1.5,1,2025.05.07.,2.0,굳이... 평점이 왜 높은지 모르겠는... 밥도 설익고 식었고..국물이 그렇게 ...,N/A
3,순대일번지,순대,821235252,리브흐,21,47,5,0,2025.03.20.,5.0,가게이름이 납득이갑니다안짜고요 양많아요양념으로 범벅하는 순대국 아니라 좋아요,N/A
4,순대일번지,순대,821235252,마르스,32,80,3.9,23,2025.02.27.,4.0,순대국특 먹었어요.다대기 들어있으니까 일단 먹어보고 싱거우면 새우젖으로 간해서 드세...,//img1.kakaocdn.net/cthumb/local/C280x280.q50/...
5,순대일번지,순대,821235252,여행 지킴이,32,116,4.7,1,2025.02.10.,5.0,여행타임tv를 운영하는 크리에이터입니다서울시내 순대국집 100군데를 다녀본바로 여긴...,//img1.kakaocdn.net/cthumb/local/C280x280.q50/...
6,순대일번지,순대,821235252,인혜,3,1,5,0,2025.02.08.,5.0,연남동에서 순대국 처음 먹어봤는데 생각 이상으로 국물이 깔끔하고 고기도 많았습니다!...,//img1.kakaocdn.net/cthumb/local/C280x280.q50/...
7,순대일번지,순대,821235252,동환,21,29,4.6,2,2025.02.08.,5.0,주에 최소 한번은 순대국 수혈해야 살수있는사람인데집근처에 있길래 한번가봤는데 너무 ...,N/A
8,순대일번지,순대,821235252,박덕동 곽종필,8,6,3.8,0,2025.02.07.,3.0,김치가 너무 달고 순대도너무적었음,N/A
9,순대일번지,순대,821235252,맥아리장군,16,21,1.8,1,2025.01.04.,1.0,제가 딱 한마디로 표현하겠습니다 여기는 순대국을 단 한번도 안 먹어본 사장님이 운영...,N/A


In [49]:
## 중복 제거 전 길이 확인
len(filtered_df)

7559

In [50]:
## 중복데이터 제거
filtered_df = filtered_df.drop_duplicates()

In [51]:
## 중복데이터 제거 후 길이 확인
len(filtered_df)

7367

In [52]:
## '분류'행의 요소들 확인
pd.options.display.max_rows = 100
print(filtered_df['분류'].value_counts())

분류
카페           848
양식           623
호프,요리주점      436
한식           410
일본식주점        397
일식           351
디저트카페        316
칵테일바         297
육류,고기        295
와인바          230
중국요리         210
일본식라면        200
이탈리안         197
피자           175
제과,베이커리      167
술집           162
커피전문점        138
중식           107
초밥,롤          97
일식집           92
태국음식          90
멕시칸,브라질       80
햄버거           77
돈까스,우동        76
치킨            71
떡볶이           65
회             60
아이스크림         59
양꼬치           58
분식            54
닭요리           53
곱창,막창         51
서점            49
해물,생선         47
국밥            40
국수            40
찌개,전골         40
베트남음식         40
샐러드           40
퓨전요리          36
실내포장마차        34
오뎅바           32
참치회           32
스페인음식         23
간식            21
동남아음식         20
한식뷔페          20
장어            20
쌈밥            20
삼겹살           20
인도음식          20
샤브샤브          19
식품            18
즉석사진          17
초콜릿           17
기업            16
퓨전일식          16
보드카페          16
순대         

In [53]:
## 제거할 항목 정리(일반음식점으로 등록되지 않은 곳)
remove_category_list = ['인테리어장식판매', '스포츠용품' ]  # '분류'열 기준
remove_store_list = ['연남스페이스']    # '사업장명'

In [54]:
## '분류'열 기준 불필요 항목 제거
for s in remove_category_list:
    filtered_df = filtered_df[filtered_df['분류'] != s]
len(filtered_df)

7362

In [55]:
## '사업장명'열 기준 불필요 항목 제거
for s in remove_store_list:
    filtered_df = filtered_df[filtered_df['사업장명'] != s]
len(filtered_df)

7360

In [56]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7360 entries, 0 to 7558
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사업장명    7360 non-null   object
 1   분류      7360 non-null   object
 2   카카오맵ID  7360 non-null   object
 3   리뷰어이름   7360 non-null   object
 4   리뷰어레벨   7360 non-null   object
 5   후기개수    7360 non-null   object
 6   별점평균    7360 non-null   object
 7   팔로워수    7360 non-null   object
 8   작성일     7360 non-null   object
 9   별점      7360 non-null   object
 10  리뷰내용    7360 non-null   object
 11  사진URL   7360 non-null   object
dtypes: object(12)
memory usage: 747.5+ KB


In [44]:
## 기본정제된 DF(filtered_df) csv 파일로 저장
filtered_df.to_csv("kakaomap_yeonnam_reviews_final.csv", index=False, encoding="utf-8-sig")
